In [548]:
import pandas as pd

import sql_functions as sf

In [549]:
schema = 'organic_africa' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = sf.get_engine()

In [550]:
pur_tables = {}
pur_tables['OGPur__OGMB'] = sf.get_dataframe(f'SELECT * FROM {schema}."OGPur__OGMB"')
pur_tables['OG_Pur__OGOrg_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."OG_Pur__OGOrg_1"')
pur_tables['wc_pur__WCOrg_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_pur__WCOrg_1"')
pur_tables['wc_pur_deviations__WCOrg_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_pur_deviations__WCOrg_1"')
pur_tables['wc_purchases_bayoba_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_purchases_bayoba_1"')
pur_tables['wc_purchases_deviations_bayoba_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_purchases_deviations_bayoba_1"')

In [551]:
del pur_tables['OGPur__OGMB']['Input_recovery']
del pur_tables['OG_Pur__OGOrg_1']['Input_recovery']

In [552]:
# column names lower case

for df in pur_tables.values():
    #print(df)
    df.columns = map(str.lower, df.columns)

pur_tables['OGPur__OGMB'].columns

Index(['id', 'og_nr', 'name_verified', 'date_of_purchase', 'receipt_nr',
       'product', 'grade', 'amount_in_kg', 'price_per_kg', 'total_payment_usd',
       'buyer', 'agr_year', 'comment', 'entered_by', 'source'],
      dtype='object')

In [553]:
pur_tables['OGPur__OGMB'].loc[pur_tables['OGPur__OGMB']['date_of_purchase']=='037/2021']
# 4097

,id,og_nr,name_verified,date_of_purchase,receipt_nr,product,grade,amount_in_kg,price_per_kg,total_payment_usd,buyer,agr_year,comment,entered_by,source
4097,6657,18493.0,False,037/2021,6091.0,Rosella (subdariffa) petals (MB),A,11.0,0.85,9.0,Philip,2021.0,None,Nyasha,OGOrg


In [554]:
pur_tables['OGPur__OGMB']['date_of_purchase'] = pur_tables['OGPur__OGMB'].iloc[4097]['date_of_purchase'].replace('037/2021', '03/07/2021')
pur_tables['OGPur__OGMB'].iloc[4097]['date_of_purchase']

'03/07/2021'

In [555]:
# Function to pad/fill to short dates like 5/3/18 to 05/03/2018

def str_date_pad(val):
    
    if type(val) is str and len(val) < 10: 

        date_splitted = val.split('/')
        print(date_splitted)
        if len(date_splitted[1]) < 2 : 
            date_splitted[1] = '0' + date_splitted[1]

        if (len(date_splitted[2]) < 4) and (int(date_splitted[2]) < 24) : 
            date_splitted[2] = '20' + date_splitted[2]
        if len(date_splitted[2]) < 4 and int(date_splitted[2]) > 23 : 
            date_splitted[2] = '19' + date_splitted[2]
    
        val = '/'.join(date_splitted)
        #print(val)
        return val
    
    else:
        return val

In [556]:
# Date_of_purchase to_datetime
pur_tables['OGPur__OGMB']['date_of_purchase'] = pur_tables['OGPur__OGMB']['date_of_purchase'].apply(str_date_pad)

In [557]:
pur_tables['OGPur__OGMB']['date_of_purchase'] = pd.to_datetime(pur_tables['OGPur__OGMB']['date_of_purchase'])

In [558]:
pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'])
pur_tables['wc_pur__WCOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_pur__WCOrg_1']['date_of_purchase'])
pur_tables['wc_pur_deviations__WCOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_pur_deviations__WCOrg_1']['date_of_purchase'])
pur_tables['wc_purchases_bayoba_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_purchases_bayoba_1']['date_of_purchase'])
pur_tables['wc_purchases_deviations_bayoba_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_purchases_deviations_bayoba_1']['date_of_purchase'])

/var/folders/x5/v1d8l9v94679fhr8vd07ys9c0000gn/T/ipykernel_13287/127177431.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'])


#### Concat

In [559]:
for name, df in pur_tables.items():
    print(name)
    print(df.shape)

OGPur__OGMB
(6612, 15)
OG_Pur__OGOrg_1
(4144, 15)
wc_pur__WCOrg_1
(2731, 15)
wc_pur_deviations__WCOrg_1
(464, 15)
wc_purchases_bayoba_1
(10451, 15)
wc_purchases_deviations_bayoba_1
(650, 15)


In [560]:
concated_pur = pd.concat(pur_tables)
concated_pur.dtypes

id                            int64
og_nr                       float64
name_verified                  bool
date_of_purchase     datetime64[ns]
receipt_nr                   object
product                      object
grade                        object
amount_in_kg                float64
price_per_kg                float64
total_payment_usd           float64
buyer                        object
agr_year                     object
comment                      object
entered_by                   object
source                       object
wc_nr                       float64
dtype: object

#### Further Cleaning
- Receipt_Nr to_numeric

In [561]:
# concated_pur['receipt_nr']= pd.to_numeric(concated_pur['receipt_nr'])
# Unable to parse string "1753A" at position 8885

concated_pur['receipt_nr'] = concated_pur.iloc[8885]['receipt_nr'].replace('1753A', '1753')

concated_pur.iloc[8885]['receipt_nr']

'1753'

In [562]:
concated_pur['receipt_nr']= pd.to_numeric(concated_pur['receipt_nr'])

In [563]:
type(concated_pur['wc_nr'])

pandas.core.series.Series

In [564]:
wc = concated_pur.loc[pd.notna(concated_pur['wc_nr'])==True,'wc_nr']
for nr in wc: 
    if str(nr).endswith('.0')==False: print(nr)

0.268


In [565]:
0.268
concated_pur['wc_nr'] = concated_pur['wc_nr'].replace('0.268', '268.0')

In [566]:
# Correcting outliers

concated_pur['agr_year'] = concated_pur['agr_year'].replace('2013/14', '2013')
concated_pur['agr_year'] = concated_pur['agr_year'].replace('2014/15', '2014')
concated_pur['agr_year'] = concated_pur['agr_year'].replace('2021q', '2021')

In [567]:
concated_pur['agr_year'] = pd.to_datetime(concated_pur['agr_year'])
#concated_pur['agr_year'] = concated_pur['agr_year'].dt.year

In [568]:
concated_pur['agr_year'] = concated_pur['agr_year'].dt.strftime('%Y')
#concated_pur['agr_year'] = pd.to_datetime(concated_pur['agr_year']).dt.year

In [569]:
display(concated_pur.dtypes)
display(concated_pur.shape)
display(concated_pur.head())

id                            int64
og_nr                       float64
name_verified                  bool
date_of_purchase     datetime64[ns]
receipt_nr                    int64
product                      object
grade                        object
amount_in_kg                float64
price_per_kg                float64
total_payment_usd           float64
buyer                        object
agr_year                     object
comment                      object
entered_by                   object
source                       object
wc_nr                       float64
dtype: object

(25052, 16)

id    og_nr  name_verified date_of_purchase  receipt_nr  \
OGPur__OGMB 0  1576  13232.0          False       2021-03-07        1753   
            1  1577  13237.0          False       2021-03-07        1753   
            2  1578  13241.0          False       2021-03-07        1753   
            3  1579  13105.0          False       2021-03-07        1753   
            4  1580  13032.0          False       2021-03-07        1753   

                                        product grade  amount_in_kg  \
OGPur__OGMB 0  Rosella (subdariffa) petals (MB)     A        15.260   
            1  Rosella (subdariffa) petals (MB)     A        74.555   
            2  Rosella (subdariffa) petals (MB)     A        61.855   
            3  Rosella (subdariffa) petals (MB)     A         6.955   
            4  Rosella (subdariffa) petals (MB)     A         7.885   

               price_per_kg  total_payment_usd   buyer agr_year comment  \
OGPur__OGMB 0          0.85               13.0  Philip     1970    None   
            1          0.85               63.0  Philip     1970    None   
            2          0.85               53.0  Philip     1970    None   
            3          0.85                6.0  Philip     1970    None   
            4          0.85                7.0  Philip     1970    None   

              entered_by source  wc_nr  
OGPur__OGMB 0     Nyasha  OGOrg    NaN  
            1     Nyasha  OGOrg    NaN  
            2     Nyasha  OGOrg    NaN  
            3     Nyasha  OGOrg    NaN  
            4     Nyasha  OGOrg    NaN

##### check duplicates and unique columns

In [570]:
# Looking for Duplicates
concated_pur.duplicated().value_counts()

# NO Duplicates

False    25052
dtype: int64

In [571]:
# sum of null values each column
concated_pur.isnull().sum()

# comment 24300 nulls

id                       0
og_nr                14321
name_verified            0
date_of_purchase       181
receipt_nr               0
product                 14
grade                13825
amount_in_kg             3
price_per_kg            25
total_payment_usd        3
buyer                   16
agr_year                16
comment              24300
entered_by             132
source                   2
wc_nr                11015
dtype: int64

In [572]:
concated_pur[['og_nr']].squeeze().is_unique

False

In [573]:
concated_pur['product'] = concated_pur['product'].replace({ 'Rosella (subdariffa) petals (MB)': 'Rosella (subdariffa) petals (org)',
                                                            'Rosella (subdariffa) seed (MB)': 'Rosella (subdariffa) seed (org)',
                                                            'Devil’s Claw plant part (org) wet': 'Devil’s Claw plant part (org)',
                                                            'kalahari melon seed (org)': 'Kalahari melon seed (org)'})

concated_pur['grade'] = concated_pur['grade'].replace(' ', None)

In [574]:
for col in ['name_verified','receipt_nr', 'entered_by', 'buyer', 'grade','product']:
    display(col)
    display(concated_pur[col].value_counts())

'name_verified'

False    17456
True      7596
Name: name_verified, dtype: int64

'receipt_nr'

1753    25052
Name: receipt_nr, dtype: int64

'entered_by'

...

Daniel             5146
Douglas            4227
Islum              2944
Philip             2404
Charles            2044
Wiseman            1528
Memory             1381
Andrew              900
Misheck             855
Ronald              755
Kennedy             703
Doreen              643
Pedzisai            642
Fungai              374
Hamid               193
Hadley              141
Nyasha              116
Nyasha Poto-Joe      28
Mandeya               9
d                     1
Meki Saizi            1
Katinka               1
Name: buyer, dtype: int64

'grade'

A    11187
B       23
C       15
Name: grade, dtype: int64

'product'

Baobab fruit whole (org)               9231
Rosella (subdariffa) petals (org)      7938
Gotu kola leaves (org)                 2086
Bird's Eye Chilli fruit whole (org)    1173
Rosella (subdariffa) seed (org)        1080
Baobab whole fruit (con)                650
Devil’s Claw root (org)                 519
Paprika fruit whole (org)               384
Devil’s Claw root (con)                 314
Trichillia emetica                      246
Ximenia Americana seed (org)            240
Baobab pulp and seed (org)              216
Strophanthus (kombe) seed (con)         163
Kalahari melon seed (org)               145
Ximenia Caffra Seed (org)               126
Gotu kola leaves (con)                  116
Marula seed (org)                        95
Devil’s Claw plant part (org)            78
Trichillia emetica (org)                 68
Marula Kennels                           41
Strophanthus (kombe) seed (org)          40
Devil’s Claw plant part (con)            32
Rosella petals (con)            

In [575]:
concated_pur

id    og_nr  name_verified  \
OGPur__OGMB                      0    1576  13232.0          False   
                                 1    1577  13237.0          False   
                                 2    1578  13241.0          False   
                                 3    1579  13105.0          False   
                                 4    1580  13032.0          False   
...                                    ...      ...            ...   
wc_purchases_deviations_bayoba_1 645   646      NaN          False   
                                 646   647      NaN          False   
                                 647   648      NaN          False   
                                 648   649      NaN          False   
                                 649   650      NaN          False   

                                     date_of_purchase  receipt_nr  \
OGPur__OGMB                      0         2021-03-07        1753   
                                 1         2021-03-07        1753   
                                 2         2021-03-07        1753   
                                 3         2021-03-07        1753   
                                 4         2021-03-07        1753   
...                                               ...         ...   
wc_purchases_deviations_bayoba_1 645       2015-08-07        1753   
                                 646       2015-08-04        1753   
                                 647       2015-08-31        1753   
                                 648       2015-08-31        1753   
                                 649       2015-09-10        1753   

                                                                product grade  \
OGPur__OGMB                      0    Rosella (subdariffa) petals (org)     A   
                                 1    Rosella (subdariffa) petals (org)     A   
                                 2    Rosella (subdariffa) petals (org)     A   
                                 3    Rosella (subdariffa) petals (org)     A   
                                 4    Rosella (subdariffa) petals (org)     A   
...                                                                 ...   ...   
wc_purchases_deviations_bayoba_1 645           Baobab whole fruit (con)  None   
                                 646           Baobab whole fruit (con)  None   
                                 647           Baobab whole fruit (con)  None   
                                 648           Baobab whole fruit (con)  None   
                                 649           Baobab whole fruit (con)  None   

                                      amount_in_kg  price_per_kg  \
OGPur__OGMB                      0          15.260          0.85   
                                 1          74.555          0.85   
                                 2          61.855          0.85   
                                 3           6.955          0.85   
                                 4           7.885          0.85   
...                                            ...           ...   
wc_purchases_deviations_bayoba_1 645        64.000          0.10   
                                 646        10.000          0.10   
                                 647        32.000          0.10   
                                 648       134.000          0.10   
                                 649        52.000          0.10   

                                      total_payment_usd    buyer agr_year  \
OGPur__OGMB                      0                 13.0   Philip     1970   
                                 1                 63.0   Philip     1970   
                                 2                 53.0   Philip     1970   
                                 3                  6.0   Philip     1970   
                                 4                  7.0   Philip     1970   
...                                                 ...      ...      ...   
wc_purchases_deviations_bayoba_1 645     

#### This purchase table has transactions of 5989 WCs and 3367 OGs = ~9300 partner farmers

From **25052** transactions with 284 transactions without og or wc numbers

In [576]:
concated_pur[concated_pur['og_nr']==14710.0]

id    og_nr  name_verified date_of_purchase  receipt_nr  \
OGPur__OGMB 1631  3853  14710.0          False       2021-03-07        1753   
            1778  4000  14710.0          False       2021-03-07        1753   
            2401  4624  14710.0           True       2021-03-07        1753   
            2509  4732  14710.0           True       2021-03-07        1753   

                                            product grade  amount_in_kg  \
OGPur__OGMB 1631  Rosella (subdariffa) petals (org)     A          58.5   
            1778    Rosella (subdariffa) seed (org)     A          38.0   
            2401  Rosella (subdariffa) petals (org)     A           5.1   
            2509    Rosella (subdariffa) seed (org)     A           4.0   

                  price_per_kg  total_payment_usd   buyer agr_year comment  \
OGPur__OGMB 1631           8.7              415.0  Philip     1970    None   
            1778           2.2               83.0  Philip     1970    None   
            2401           8.8                0.0  Memory     1970    None   
            2509           2.2                8.7  Memory     1970    None   

                      entered_by source  wc_nr  
OGPur__OGMB 1631  Norman Kativhu  OGOrg    NaN  
            1778  Norman Kativhu  OGOrg    NaN  
            2401  Norman Kativhu  OGOrg    NaN  
            2509  Norman Kativhu  OGOrg    NaN

In [577]:
concated_pur['wc_nr'].duplicated().value_counts()

True     19063
False     5989
Name: wc_nr, dtype: int64

In [578]:
concated_pur['og_nr'].duplicated().value_counts()

True     21685
False     3367
Name: og_nr, dtype: int64

In [579]:
concated_pur[concated_pur['id']==7572]

,,id,og_nr,name_verified,date_of_purchase,receipt_nr,product,grade,amount_in_kg,price_per_kg,total_payment_usd,buyer,agr_year,comment,entered_by,source,wc_nr
OGPur__OGMB,1503,7572,NaN,False,2021-03-07,1753,None,None,0.0,0.0,0.0,None,NaN,None,None,OGOrg,NaN


In [580]:
nr=concated_pur[~concated_pur['wc_nr'].isna() | ~concated_pur['og_nr'].isna()].shape
nr[0]

24768

In [581]:
all=concated_pur.shape
all[0]

25052

In [582]:
all[0]-nr[0]

284

In [583]:
# 57 
concated_pur[concated_pur['total_payment_usd']<=0][['id','og_nr','wc_nr','date_of_purchase','total_payment_usd']]

id    og_nr    wc_nr  \
OGPur__OGMB                      1503  7572      NaN      NaN   
                                 2315  4538  19206.0      NaN   
                                 2336  4559  21027.0      NaN   
                                 2358  4581  19135.0      NaN   
                                 2383  4606  19608.0      NaN   
                                 2386  4609  19621.0      NaN   
                                 2401  4624  14710.0      NaN   
                                 2402  4625  21850.0      NaN   
                                 2407  4630  19663.0      NaN   
                                 2408  4631  19665.0      NaN   
                                 2427  4650  19285.0      NaN   
                                 2430  4653  21804.0      NaN   
                                 2433  4656  21804.0      NaN   
                                 2439  4662  20225.0      NaN   
                                 2440  4663  21093.0      NaN   
                                 3251  7573      NaN      NaN   
                                 4001  6562      NaN      NaN   
                                 4017  7574      NaN      NaN   
                                 4208  6768  25685.0      NaN   
                                 4297  6857  11602.0      NaN   
                                 4539  7099      NaN      NaN   
                                 4540  7100      NaN      NaN   
                                 4541  7101      NaN      NaN   
                                 4542  7102      NaN      NaN   
                                 4590  7150      NaN      NaN   
                                 4591  7151      NaN      NaN   
                                 4592  7152      NaN      NaN   
                                 4593  7153      NaN      NaN   
                                 4594  7154      NaN      NaN   
                                 5975  8540  12134.0      NaN   
                                 6075  8640  13108.0      NaN   
                                 6086  8651  13108.0      NaN   
                                 6226  8791  24305.0      NaN   
                                 6241  8806  18278.0      NaN   
                                 6242  8807  18267.0      NaN   
                                 6253  8818  18151.0      NaN   
                                 6341  8906  13034.0      NaN   
                                 6344  8909  24787.0      NaN   
                                 6345  8910  26060.0      NaN   
                                 6354  8919  18951.0      NaN   
                                 6360  8925  15985.0      NaN   
                                 6364  8929  15980.0      NaN   
                                 6366  8931  15981.0      NaN   
                                 6372  8937  18351.0      NaN   
                                 6399  8964  24393.0      NaN   
                                 6401  8966  15985.0      NaN   
                                 6432  8997  13034.0      NaN   
                                 6460  9025  19257.0      NaN   
                                 6582  9147  19257.0      NaN   
OG_Pur__OGOrg_1                  2435  4029  14046.0      NaN   
                                 2467  4061  19427.0      NaN   
wc_purchases_bayoba_1            1846  8958      NaN  41453.0   
                                 1881  8993      NaN  41483.0   
                                 1992  9104      NaN  41337.0   
wc_purchases_deviations_bayoba_1 379    380      NaN   1453.0   
                                 413    414      NaN   1483.0   
                                 523    524      NaN   1337.0   

                                      date_of_purchase  total_payment_usd  
OGPur__OGMB                      1503       2021-03-07                0.0  
                                 2315       2021-03-07                0.0  
                                 2336     

In [584]:
concated_pur[concated_pur['receipt_nr']==1753].shape[0]

25052

## Dropping unnecessary 13 rows with too many nans

In [585]:
jkl = concated_pur[concated_pur['wc_nr'].isna() & concated_pur['og_nr'].isna()]
jkl = jkl[jkl['total_payment_usd']<=0]
jkl

id  og_nr  name_verified date_of_purchase  receipt_nr  \
OGPur__OGMB 1503  7572    NaN          False       2021-03-07        1753   
            3251  7573    NaN          False       2021-03-07        1753   
            4001  6562    NaN          False       2021-03-07        1753   
            4017  7574    NaN          False       2021-03-07        1753   
            4539  7099    NaN          False       2021-03-07        1753   
            4540  7100    NaN          False       2021-03-07        1753   
            4541  7101    NaN          False       2021-03-07        1753   
            4542  7102    NaN          False       2021-03-07        1753   
            4590  7150    NaN          False       2021-03-07        1753   
            4591  7151    NaN          False       2021-03-07        1753   
            4592  7152    NaN          False       2021-03-07        1753   
            4593  7153    NaN          False       2021-03-07        1753   
            4594  7154    NaN          False       2021-03-07        1753   

                 product grade  amount_in_kg  price_per_kg  total_payment_usd  \
OGPur__OGMB 1503    None  None           0.0           0.0                0.0   
            3251    None  None           0.0           0.0                0.0   
            4001    None  None           0.0           0.0                0.0   
            4017    None  None           0.0           0.0                0.0   
            4539    None  None           0.0           0.0                0.0   
            4540    None  None           0.0           0.0                0.0   
            4541    None  None           0.0           0.0                0.0   
            4542    None  None           0.0           0.0                0.0   
            4590    None  None           0.0           0.0                0.0   
            4591    None  None           0.0           0.0                0.0   
            4592    None  None           0.0           0.0                0.0   
            4593    None  None           0.0           0.0                0.0   
            4594    None  None           0.0           0.0                0.0   

                 buyer agr_year comment entered_by source  wc_nr  
OGPur__OGMB 1503  None      NaN    None       None  OGOrg    NaN  
            3251  None      NaN    None       None  OGOrg    NaN  
            4001  None      NaN    None       None  OGOrg    NaN  
            4017  None      NaN    None       None  OGOrg    NaN  
            4539  None      NaN    None       None  OGOrg    NaN  
            4540  None      NaN    None       None  OGOrg    NaN  
            4541  None      NaN    None       None  OGOrg    NaN  
            4542  None      NaN    None       None   None    NaN  
            4590  None      NaN    None       None  OGOrg    NaN  
            4591  None      NaN    None       None  OGOrg    NaN  
            4592  None      NaN    None       None  OGOrg    NaN  
            4593  None      NaN    None       None  OGOrg    NaN  
            4594  None      NaN    None       None  OGOrg    NaN

In [586]:
multi_index_nan = jkl[jkl['total_payment_usd']<=0].index
multi_index_nan

# [1503, 3251, 4001, 4017, 4539, 4540, 4541, 4542, 4590, 4591, 4592, 4593, 4594]

MultiIndex([('OGPur__OGMB', 1503),
            ('OGPur__OGMB', 3251),
            ('OGPur__OGMB', 4001),
            ('OGPur__OGMB', 4017),
            ('OGPur__OGMB', 4539),
            ('OGPur__OGMB', 4540),
            ('OGPur__OGMB', 4541),
            ('OGPur__OGMB', 4542),
            ('OGPur__OGMB', 4590),
            ('OGPur__OGMB', 4591),
            ('OGPur__OGMB', 4592),
            ('OGPur__OGMB', 4593),
            ('OGPur__OGMB', 4594)],
           )

In [587]:
drop_list = [1503, 3251, 4001, 4017, 4539, 4540, 4541, 4542, 4590, 4591, 4592, 4593, 4594]

In [588]:
concated_pur.drop(concated_pur.index[drop_list], inplace=True)

In [589]:
concated_pur[concated_pur['wc_nr'].isna() & concated_pur['og_nr'].isna()]


id  og_nr  name_verified date_of_purchase  \
OGPur__OGMB           4641    7201    NaN          False       2021-03-07   
                      5829    8393    NaN          False       2021-03-07   
                      5830    8394    NaN          False       2021-03-07   
                      5831    8395    NaN          False       2021-03-07   
                      5838    8402    NaN          False       2021-03-07   
...                            ...    ...            ...              ...   
wc_purchases_bayoba_1 10132  17344    NaN          False       2022-06-16   
                      10236  17448    NaN          False       2022-06-09   
                      10238  17450    NaN          False       2022-06-09   
                      10406  17618    NaN          False       2021-10-27   
                      10423  17635    NaN          False       2021-10-27   

                             receipt_nr                            product  \
OGPur__OGMB           4641         1753  Rosella (subdariffa) petals (org)   
                      5829         1753  Rosella (subdariffa) petals (org)   
                      5830         1753  Rosella (subdariffa) petals (org)   
                      5831         1753  Rosella (subdariffa) petals (org)   
                      5838         1753  Rosella (subdariffa) petals (org)   
...                                 ...                                ...   
wc_purchases_bayoba_1 10132        1753           Baobab fruit whole (org)   
                      10236        1753           Baobab fruit whole (org)   
                      10238        1753           Baobab fruit whole (org)   
                      10406        1753           Baobab fruit whole (org)   
                      10423        1753           Baobab fruit whole (org)   

                            grade  amount_in_kg  price_per_kg  \
OGPur__OGMB           4641      A         34.00          0.85   
                      5829      A         10.50          0.85   
                      5830      A         26.95          0.85   
                      5831      A          8.45          0.85   
                      5838      A          6.40          0.85   
...                           ...           ...           ...   
wc_purchases_bayoba_1 10132  None        112.00          0.14   
                      10236  None        720.00          0.14   
                      10238  None        324.00          0.14   
                      10406  None        241.00          0.12   
                      10423  None        111.00          0.12   

                             total_payment_usd    buyer agr_year comment  \
OGPur__OGMB           4641               20.00   Ronald     1970    None   
                      5829                4.00  Kennedy     1970    None   
                      5830               18.00  Kennedy     1970    None   
                      5831                5.00  Kennedy     1970    None   
                      5838                5.00  Kennedy     1970    None   
...                                        ...      ...      ...     ...   
wc_purchases_bayoba_1 10132              15.68    Islum     2022    None   
                      10236             100.80    Islum     2022    None   
                      10238              45.36    Islum     2022    None   
                      10406              28.90  Douglas     2021    None   
                      10423              13.30  Douglas     2021    None   

                            entered_by  source  wc_nr  
OGPur__OGMB           4641      Nyasha   OGOrg    NaN  
                      5829      Nyasha   OGOrg    NaN  
                      5830      Nyasha   OGOrg    NaN  
                      5831      Nyasha   OGOrg    NaN  
                      5838      Nyasha   OGOrg    NaN  
...                                ...     ...    ...  
wc_purchases_bayoba_1 10132     Nyasha  BAyoba    NaN  
                      10236 

In [590]:
concated_pur[concated_pur['total_payment_usd']<=0]

id    og_nr  name_verified  \
OGPur__OGMB                      2315  4538  19206.0           True   
                                 2336  4559  21027.0           True   
                                 2358  4581  19135.0           True   
                                 2383  4606  19608.0           True   
                                 2386  4609  19621.0           True   
                                 2401  4624  14710.0           True   
                                 2402  4625  21850.0           True   
                                 2407  4630  19663.0           True   
                                 2408  4631  19665.0           True   
                                 2427  4650  19285.0          False   
                                 2430  4653  21804.0          False   
                                 2433  4656  21804.0          False   
                                 2439  4662  20225.0          False   
                                 2440  4663  21093.0          False   
                                 4208  6768  25685.0          False   
                                 4297  6857  11602.0          False   
                                 5975  8540  12134.0          False   
                                 6075  8640  13108.0          False   
                                 6086  8651  13108.0          False   
                                 6226  8791  24305.0          False   
                                 6241  8806  18278.0          False   
                                 6242  8807  18267.0          False   
                                 6253  8818  18151.0          False   
                                 6341  8906  13034.0          False   
                                 6344  8909  24787.0          False   
                                 6345  8910  26060.0          False   
                                 6354  8919  18951.0          False   
                                 6360  8925  15985.0          False   
                                 6364  8929  15980.0          False   
                                 6366  8931  15981.0          False   
                                 6372  8937  18351.0          False   
                                 6399  8964  24393.0          False   
                                 6401  8966  15985.0          False   
                                 6432  8997  13034.0          False   
                                 6460  9025  19257.0          False   
                                 6582  9147  19257.0          False   
OG_Pur__OGOrg_1                  2435  4029  14046.0          False   
                                 2467  4061  19427.0          False   
wc_purchases_bayoba_1            1846  8958      NaN          False   
                                 1881  8993      NaN          False   
                                 1992  9104      NaN          False   
wc_purchases_deviations_bayoba_1 379    380      NaN          False   
                                 413    414      NaN          False   
                                 523    524      NaN          False   

                                      date_of_purchase  receipt_nr  \
OGPur__OGMB                      2315       2021-03-07        1753   
                                 2336       2021-03-07        1753   
                                 2358       2021-03-07        1753   
                                 2383       2021-03-07        1753   
                                 2386       2021-03-07        1753   
                                 2401       2021-03-07        1753   
                                 2402       2021-03-07        1753   
                                 2407       2021-03-07        1753   
                                 2408       2021-03-07        1753   
                                 2427       2021-03-07        1753   
                                 2430       2021-03-07        1753   
     

In [591]:
concated_pur[['id']].squeeze().is_unique

False

In [592]:
table_name = 'all_purchase'

if engine!=None:
    try:
        concated_pur.to_sql(name=table_name, # Name of SQL table
                                con=engine, # Engine or connection
                                if_exists='replace', # Drop the table before inserting new values 
                                schema=schema, # Use schmea that was defined earlier
                                index=False, # Write DataFrame index as a column
                                chunksize=5000, # Specify the number of rows in each batch to be written at a time
                                method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The all_purchase table was imported successfully.
